# `CLOUD.jl` - 2D linear advection example

In [42]:
using OrdinaryDiffEq
using LinearAlgebra
using TimerOutputs
using CLOUD;

Define the physical problem

In [43]:
θ = π/4 # wave angle
a = sqrt(2) # wave speed
A = 1.0  # amplitude
L = 1.0  # domain length
k = (2*π/L, 2*π/L)  # wave number
T = 1.0/(a*max(abs(cos(θ)),abs(cos(θ)))) # end time for one period
initial_data = InitialDataSine(A,k)
conservation_law = linear_advection_equation((a*cos(θ),a*sin(θ)),λ=1.0);

Set discretization parameters

In [44]:
M = 4
p = 4
p_map = 4
form = SplitConservationForm()
integrator = CarpenterKennedy2N54()
n_s = 10
dt_scale=0.05
dt = dt_scale*(L/M)/(norm(a)*(2*p+1));

Set up a `DGSEM` scheme with LG quadrature on quadrilaterals of degree `p` with `M` edges in each direction

In [45]:
reference_approximation =ReferenceApproximation(
    DGSEM(p), Quad(),
    volume_quadrature_rule=LGQuadrature(),
    facet_quadrature_rule=LGQuadrature(),
    mapping_degree=p_map,
    N_plot=ceil(Int,50/M))

mesh = warp_mesh(uniform_periodic_mesh(
    reference_approximation.reference_element, 
    ((0.0,L),(0.0,L)), (M,M)), 
    reference_approximation.reference_element, 0.1)

spatial_discretization = SpatialDiscretization(mesh, 
    reference_approximation)

results_path = save_project(conservation_law,
     spatial_discretization, initial_data, form, (0.0, T), Lazy(),
     "results/advection_2d_dgsem/", overwrite=true, clear=true)

visualize(spatial_discretization, results_path, "mesh.pdf", 
    plot_volume_nodes=false, grid_lines=true)

Run the solver and plot the solution

In [46]:
ode_problem = semidiscretize(load_project(results_path)...)
save_solution(ode_problem.u0, 0.0, results_path, 0)

for t in 1:Threads.nthreads()
    reset_timer!(get_timer(string("thread_timer_",t)))
end

sol = solve(ode_problem, integrator, adaptive=false,
    dt=dt, save_everystep=false,
    callback=save_callback(results_path, ceil(Int, T/(dt*n_s))))

to = merge(Tuple(get_timer(string("thread_timer_",t)) 
    for t in 1:Threads.nthreads())...)
print_timer(to)

save_solution(last(sol.u), last(sol.t), results_path, "final")
plotter = Plotter(spatial_discretization, results_path)
visualize(last(sol.u), plotter, "approx.pdf", u_range=[-1.0,1.0], contours=25,
    label="U^h(\\mathbf{x},t)")

 ────────────────────────────────────────────────────────────────────────────────
                                        Time                    Allocations      
                               ───────────────────────   ────────────────────────
       Tot / % measured:          227μs / 1143236.4%      14.0KiB / 18288549.7%  

 Section               ncalls     time    %tot     avg     alloc    %tot      avg
 ────────────────────────────────────────────────────────────────────────────────
 eval residual          81.5k    1.12s   43.2%  13.8μs    774MiB   31.1%  9.72KiB
   volume terms         81.5k    730ms   28.1%  8.96μs    500MiB   20.1%  6.28KiB
   facet terms          81.5k    168ms    6.5%  2.06μs    173MiB    6.9%  2.17KiB
   mass matrix solve    81.5k   94.5ms    3.6%  1.16μs   39.8MiB    1.6%     512B
 eval flux diff         81.5k    905ms   34.8%  11.1μs   0.99GiB   40.7%  12.7KiB
 extrapolate solution   81.5k    182ms    7.0%  2.23μs    142MiB    5.7%  1.78KiB
 eval numerical

Calculate the integral L<sup>2</sup> error norm and conservation/energy error

In [47]:
error_analysis = ErrorAnalysis(results_path, conservation_law, 
    spatial_discretization)
conservation_analysis = PrimaryConservationAnalysis(results_path, 
    conservation_law, spatial_discretization)
energy_analysis = EnergyConservationAnalysis(results_path, 
    conservation_law, spatial_discretization)

println("L2 error")
println(analyze(error_analysis, last(sol.u), initial_data)...)
println("Conservation (initial/final/diff):")
println(analyze(conservation_analysis)...)
println("Energy (initial/final/diff):")
println(analyze(energy_analysis)...)

L2 error
0.000683954884588324
Conservation (initial/final/diff):
[-1.108594540819019e-8][-1.1085945919933615e-8][-5.117434254131581e-16]
Energy (initial/final/diff):
[0.12499998811286066][0.12497956180878655][-2.0426304074103174e-5]


Set up a `DGMulti` scheme on triangles of degree `p` with `M` edges in each direction

In [51]:
reference_approximation = ReferenceApproximation(
    DGMulti(p), Tri(),
    mapping_degree=p_map,
    N_plot=ceil(Int,50/M))

mesh = warp_mesh(uniform_periodic_mesh(
    reference_approximation.reference_element, 
    ((0.0,L),(0.0,L)), (M,M)), 
    reference_approximation.reference_element, 0.1)

spatial_discretization = SpatialDiscretization(mesh, 
    reference_approximation)

results_path = save_project(conservation_law,
     spatial_discretization, initial_data, form, (0.0, T), Lazy(),
     "results/advection_2d_dgmulti/", overwrite=true, clear=true)

visualize(spatial_discretization, results_path, "mesh.pdf", 
    plot_volume_nodes=true, grid_lines=false)

UndefVarError: UndefVarError: d not defined

Run the solver and plot the solution

In [49]:
ode_problem = semidiscretize(load_project(results_path)...)
save_solution(ode_problem.u0, 0.0, results_path, 0)

for t in 1:Threads.nthreads()
    reset_timer!(get_timer(string("thread_timer_",t)))
end

sol = solve(ode_problem, integrator, adaptive=false,
    dt=dt, save_everystep=false,
    callback=save_callback(results_path, ceil(Int, T/(dt*n_s))))

to = merge(Tuple(get_timer(string("thread_timer_",t)) 
    for t in 1:Threads.nthreads())...)
print_timer(to)

save_solution(last(sol.u), last(sol.t), results_path, "final")
plotter = Plotter(spatial_discretization, results_path)
visualize(last(sol.u), plotter, "approx.pdf", u_range=[-1.0,1.0], contours=25,
    label="U^h(\\mathbf{x},t)")

 ────────────────────────────────────────────────────────────────────────────────
                                        Time                    Allocations      
                               ───────────────────────   ────────────────────────
       Tot / % measured:          214μs / 1147221.6%      14.0KiB / 18288549.7%  

 Section               ncalls     time    %tot     avg     alloc    %tot      avg
 ────────────────────────────────────────────────────────────────────────────────
 eval residual          81.5k    973ms   39.7%  11.9μs    774MiB   31.1%  9.72KiB
   volume terms         81.5k    611ms   24.9%  7.49μs    500MiB   20.1%  6.28KiB
   facet terms          81.5k    162ms    6.6%  1.99μs    173MiB    6.9%  2.17KiB
   mass matrix solve    81.5k   91.5ms    3.7%  1.12μs   39.8MiB    1.6%     512B
 eval flux diff         81.5k    960ms   39.2%  11.8μs   0.99GiB   40.7%  12.7KiB
 eval numerical flux    81.5k    167ms    6.8%  2.05μs    212MiB    8.5%  2.66KiB
 extrapolate so

Calculate the integral L<sup>2</sup> error norm and conservation/energy error

In [50]:
error_analysis = ErrorAnalysis(results_path, conservation_law, 
    spatial_discretization)
conservation_analysis = PrimaryConservationAnalysis(results_path, 
    conservation_law, spatial_discretization)
energy_analysis = EnergyConservationAnalysis(results_path, 
    conservation_law, spatial_discretization)

println("L2 error")
println(analyze(error_analysis, last(sol.u), initial_data)...)
println("Conservation (initial/final/diff):")
println(analyze(conservation_analysis)...)
println("Energy (initial/final/diff):")
println(analyze(energy_analysis)...)

L2 error
0.000683954884588324
Conservation (initial/final/diff):
[-1.108594540819019e-8][-1.1085945919933615e-8][-5.117434254131581e-16]
Energy (initial/final/diff):
[0.12499998811286066][0.12497956180878655][-2.0426304074103174e-5]
